# 1. Import Libraries

Importing all necessary libraries: `requests` for the API call, `pandas` to check our data, `json` to parse it, `openpyxl` to save to Excel, and `re` for accurate text matching.

In [19]:
import requests
import pandas as pd
import json
from openpyxl import Workbook
import re  # Import the regular expressions library

# 2. Download Data (ONCE)

Instead of running a separate API server, I'll use the direct cloud URL for the JSON data. I will download the data *one time* and store it in a global variable `jobs_data`. This is much more efficient.

In [20]:
# This is the NEW, correct URL for the data file
api_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/module%201/Accessing%20Data%20Using%20APIs/jobs.json"

print("Downloading job data...")
response_api = requests.get(api_url)

if response_api.ok:
    jobs_data = response_api.json()
    print(f"Data downloaded successfully. Found {len(jobs_data)} job postings.")
else:
    print(f"Failed to download data. Status code: {response_api.status_code}")
    jobs_data = [] # Create empty list to avoid errors

Data downloaded successfully. Found 27005 job postings.


# 3. Define Helper Functions

These helper functions will search the `jobs_data` we already downloaded.

In [22]:
def get_number_of_jobs_T(technology):
    number_of_jobs = 0

    # Create a precise regex pattern to find the technology
    # re.IGNORECASE makes it case-insensitive
    pattern = r"\b" + re.escape(technology) + r"\b"

    # Special cases for C++ and C#
    if technology == 'C++':
        pattern = r"C\+\+" # Need to escape the +
    elif technology == 'C#':
        pattern = r"C#"

    for job in jobs_data:
        key = job.get('Key Skills')
        if key is not None:
            # Use re.search for an accurate, case-insensitive match
            if re.search(pattern, key, re.IGNORECASE):
                number_of_jobs += 1

    return technology, number_of_jobs

def get_number_of_jobs_L(location):
    number_of_jobs = 0
    for job in jobs_data:
        loc = job.get('Location')
        # Location matching can be simpler, 'in' is fine
        if loc is not None and location.lower() in loc.lower():
            number_of_jobs += 1

    return location, number_of_jobs

# 4. Collect Location Data and Save to Excel

Now I'll loop through the locations and use the new helper function to save the job counts.

In [23]:
locations = ['Los Angeles', 'New York', 'San Francisco', 'Washington DC', 'Seattle', 'Austin', 'Detroit']

wb1 = Workbook()
ws1 = wb1.active
ws1.append(['Location', 'Number of Jobs'])

print("Collecting job data by location...")
for loc in locations:
    ws1.append(get_number_of_jobs_L(loc))

wb1.save('2.a-job-postings (Collected from API).xlsx')
wb1.close()
print("Saved '2.a-job-postings (Collected from API).xlsx'")

Saved '2.a-job-postings (Collected from API).xlsx'


# 5. Collect Technology Data and Save to Excel

Same process for the technologies.

In [24]:
languages = ['C', 'C#', 'C++', 'Java', 'JavaScript', 'Python', 'Scala', 'Oracle', 'SQL Server', 'MySQL Server', 'PostgreSQL', 'MongoDB']

wb2 = Workbook()
ws2 = wb2.active
ws2.append(['Language', 'Number of Jobs'])

print("Collecting job data by technology...")
for lang in languages:
    ws2.append(get_number_of_jobs_T(lang))

wb2.save('2.a-job-postings-languages (Collected from API).xlsx')
wb2.close()
print("Saved '2.a-job-postings-languages (Collected from API).xlsx'")

Saved '2.a-job-postings-languages (Collected from API).xlsx'


# 6. Verify Excel Files

Finally, I'll read the two new Excel files to verify they have the correct data.

In [25]:
print("--- Location Data ---")
df_loc = pd.read_excel('2.a-job-postings (Collected from API).xlsx')
print(df_loc)

print("\n--- Technology Data ---")
df_tech = pd.read_excel('2.a-job-postings-languages (Collected from API).xlsx')
print(df_tech)

--- Location Data ---
        Location  Number of Jobs
0    Los Angeles             640
1       New York            3226
2  San Francisco             435
3  Washington DC            5316
4        Seattle            3375
5         Austin             434
6        Detroit            3945

--- Technology Data ---
        Language  Number of Jobs
0              C            1655
1             C#             526
2            C++             506
3           Java            1601
4     JavaScript            2246
5         Python            1171
6          Scala              89
7         Oracle             899
8     SQL Server             422
9   MySQL Server               0
10    PostgreSQL              86
11       MongoDB             208
